# Setup

In [ ]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Dataset_Person 
%mkdir GoogleNet
!ls fiftyone/coco-2017

/content/drive/MyDrive/Dataset_Person
info.json  raw	train  validation


# Prepare dependencies

In [ ]:
# Langsung tersimpan di drive, jadi cukup di run 1 kali saja

!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py

--2023-06-08 07:35:05--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23337 (23K) [text/plain]
Saving to: ‘transforms.py’

transforms.py       100%[===================>]  22.79K  --.-KB/s    in 0.002s  

2023-06-08 07:35:05 (12.8 MB/s) - ‘transforms.py’ saved [23337/23337]

--2023-06-08 07:35:05--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4032 (3.9K)

# Import Dependencies

In [ ]:
import torch
import torchvision
import torch.utils.data
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.coco as fouc

import utils
import transforms as T
from engine import train_one_epoch, evaluate

from PIL import Image

Migrating database to v0.21.0


INFO:fiftyone.migrations.runner:Migrating database to v0.21.0


# Prepare Dataset

In [ ]:
train_dir = "fiftyone/coco-2017/train"
test_dir = "fiftyone/coco-2017/validation"

# Folder untuk simpan model dan data hasil prediksi
save_dir = "GoogleNet"

In [ ]:
train_dataset = fo.Dataset.from_dir(
    dataset_dir = train_dir,
    dataset_type = fo.types.COCODetectionDataset,
    label_types = ["detections"],
    classes = ["person"]
)

test_dataset = fo.Dataset.from_dir(
    dataset_dir = test_dir,
    dataset_type = fo.types.COCODetectionDataset,
    label_types=["detections"],
    classes = ["person"]
)

 100% |███████████████| 3000/3000 [23.0s elapsed, 0s remaining, 151.4 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 3000/3000 [23.0s elapsed, 0s remaining, 151.4 samples/s]      


 100% |█████████████████| 500/500 [3.4s elapsed, 0s remaining, 138.9 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [3.4s elapsed, 0s remaining, 138.9 samples/s]      


In [ ]:
# Iterate over the dataset
for sample in train_dataset:
    # Get the detections
    detections = sample.ground_truth.detections
    # Filter out non-person detections
    detections = [d for d in detections if d.label == "person"]
    # Update the detections
    sample.ground_truth.detections = detections
    # Save the sample
    sample.save()

In [ ]:
# Iterate over the dataset
for sample in test_dataset:
    # Get the detections
    detections = sample.ground_truth.detections
    # Filter out non-person detections
    detections = [d for d in detections if d.label == "person"]
    # Update the detections
    sample.ground_truth.detections = detections
    # Save the sample
    sample.save()

## Object Dataset

In [ ]:
class ObjectDataset(torch.utils.data.Dataset):
  def __init__(self, root, transforms=None):
    #please define the data proses init
    self.root = root
    self.transforms = transforms

    self.img_paths = self.root.values("filepath")

    self.classes = self.root.distinct("%s.detections.label" % "ground_truth")
    if self.classes[0] != "background":
        self.classes = ["background"] + self.classes

    self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

  def __getitem__(self, idx):
    # please define the dataloader
    img_path = self.img_paths[idx]
    sample_img = self.root[img_path]
    metadata = sample_img.metadata

    img = Image.open(img_path).convert('RGB')

    boxes = []
    labels = []
    image_id = []
    area = []
    iscrowd = []

    for det in sample_img["ground_truth"].detections:
      category_id = self.labels_map_rev[det.label]
      coco_obj = fouc.COCOObject.from_label(
          det, metadata, category_id=category_id,
      )
      x, y, w, h = coco_obj.bbox
      boxes.append([x, y, x + w, y + h])
      labels.append(coco_obj.category_id)
      area.append(coco_obj.area)
      iscrowd.append(coco_obj.iscrowd)

    target = {}
    target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
    target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
    target["image_id"] = torch.as_tensor([idx])
    target["area"] = torch.as_tensor(area, dtype=torch.float32)
    target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

    if self.transforms is not None: #preprocessing dan augmentasi
      img, target = self.transforms(img, target)

    return img, target
  
  def __len__(self):
    return len(self.img_paths)


## Data Transforms

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    
    return T.Compose(transforms)

## Apply Transformations and Dataloader

In [ ]:
# Use our dataset and defined transformations
train_data = ObjectDataset(train_dataset, get_transform(train=True))
test_data = ObjectDataset(test_dataset, get_transform(train=False))

# Define training and validation data loaders
train_dataloader = torch.utils.data.DataLoader(
  train_data, batch_size=2, shuffle=True, num_workers=2,
  collate_fn=utils.collate_fn)

test_dataloader = torch.utils.data.DataLoader(
  test_data, batch_size=1, shuffle=False, num_workers=2,
  collate_fn=utils.collate_fn)

# Build Model

## GoogleNet

In [ ]:
# backbone googlenet
googlenet = torchvision.models.googlenet(weights="DEFAULT")

backbone = torch.nn.Sequential(*list(googlenet.children())[:-3]) # menghilangkan fully connected layer dan global avg pool

backbone.out_channels = 1024

anchor_generator = AnchorGenerator()

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names= ['0'], output_size=7, sampling_ratio=2)

googlenet_model = FasterRCNN(backbone, num_classes=2, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 189MB/s]


## Train and Evaluate Model

In [ ]:
def train(model, train_dataloader, test_dataloader, num_epochs=5):
    
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # move model to the right device
    model.to(device)
    
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
    for epoch in range(num_epochs):
        # train for one epoch, printing every 100 iterations
        train_one_epoch(model, optimizer, train_dataloader, device, epoch, print_freq=100)

        # update the learning rate
        lr_scheduler.step()
        
        # evaluate on the test dataset
        evaluate(model, test_dataloader, device=device)
    
    print("That's it!")

In [ ]:
train(googlenet_model, train_dataloader, test_dataloader, 10)

Epoch: [0]  [   0/1500]  eta: 4:18:16  lr: 0.000010  loss: 1.4106 (1.4106)  loss_classifier: 0.6804 (0.6804)  loss_box_reg: 0.0270 (0.0270)  loss_objectness: 0.6758 (0.6758)  loss_rpn_box_reg: 0.0275 (0.0275)  time: 10.3307  data: 0.5992  max mem: 2693
Epoch: [0]  [ 100/1500]  eta: 0:16:14  lr: 0.000509  loss: 0.7063 (1.1271)  loss_classifier: 0.0975 (0.2610)  loss_box_reg: 0.0277 (0.0467)  loss_objectness: 0.4523 (0.5956)  loss_rpn_box_reg: 0.0679 (0.2238)  time: 0.3399  data: 0.0112  max mem: 4174
Epoch: [0]  [ 200/1500]  eta: 0:11:21  lr: 0.001009  loss: 0.5353 (0.9225)  loss_classifier: 0.0907 (0.1854)  loss_box_reg: 0.0375 (0.0413)  loss_objectness: 0.3036 (0.4993)  loss_rpn_box_reg: 0.0520 (0.1966)  time: 0.3563  data: 0.0128  max mem: 4175
Epoch: [0]  [ 300/1500]  eta: 0:09:22  lr: 0.001508  loss: 0.5011 (0.8220)  loss_classifier: 0.1002 (0.1620)  loss_box_reg: 0.0613 (0.0495)  loss_objectness: 0.2250 (0.4344)  loss_rpn_box_reg: 0.0515 (0.1761)  time: 0.3612  data: 0.0116  max m

## Save Model

In [ ]:
torch.save(googlenet_model.state_dict(), save_dir + "/googlenet_ep10.pt")

In [ ]:
def convert_torch_predictions(preds, det_id, s_id, w, h, classes):
    # Convert the outputs of the torch model into a FiftyOne Detections object
    dets = []
    for bbox, label, score in zip(
        preds["boxes"].cpu().detach().numpy(), 
        preds["labels"].cpu().detach().numpy(), 
        preds["scores"].cpu().detach().numpy()
    ):
        # Parse prediction into FiftyOne Detection object
        x0,y0,x1,y1 = bbox
        coco_obj = fouc.COCOObject(det_id, s_id, int(label), [x0, y0, x1-x0, y1-y0])
        det = coco_obj.to_detection((w,h), classes)
        det["confidence"] = float(score)
        dets.append(det)
        det_id += 1
        
    detections = fo.Detections(detections=dets)
        
    return detections, det_id

def add_detections(model, torch_dataset, view, field_name="predictions"):
    # Run inference on a dataset and add results to FiftyOne
    torch.set_num_threads(1)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    model.eval()
    model.to(device)
    image_paths = torch_dataset.img_paths
    classes = torch_dataset.classes
    det_id = 0
    
    with fo.ProgressBar() as pb:
        for img, targets in pb(torch_dataset):
            # Get FiftyOne sample indexed by unique image filepath
            img_id = int(targets["image_id"][0])
            img_path = image_paths[img_id]
            sample = view[img_path]
            s_id = sample.id
            w = sample.metadata["width"]
            h = sample.metadata["height"]
            
            # Inference
            preds = model(img.unsqueeze(0).to(device))[0]
            
            detections, det_id = convert_torch_predictions(
                preds, 
                det_id, 
                s_id, 
                w, 
                h, 
                classes,
            )
            
            sample[field_name] = detections
            sample.save()

In [ ]:
add_detections(googlenet_model, test_data, test_dataset, field_name="predictions")

Using device cuda
 100% |█████████████████| 500/500 [35.7s elapsed, 0s remaining, 15.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [35.7s elapsed, 0s remaining, 15.2 samples/s]      


In [ ]:
results = fo.evaluate_detections(
    test_dataset, 
    "predictions", 
    classes=["person"], 
    eval_key="eval", 
    compute_mAP=True
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 500/500 [40.3s elapsed, 0s remaining, 12.1 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [40.3s elapsed, 0s remaining, 12.1 samples/s]      


Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 500/500 [15.5s elapsed, 0s remaining, 37.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [15.5s elapsed, 0s remaining, 37.2 samples/s]      


In [ ]:
results.mAP()

0.24516742211808992

In [ ]:
results.print_report()

              precision    recall  f1-score   support

      person       0.37      0.86      0.52      4562

   micro avg       0.37      0.86      0.52      4562
   macro avg       0.37      0.86      0.52      4562
weighted avg       0.37      0.86      0.52      4562



In [ ]:
# Export labeled dataset in COCO format
test_dataset.export(
    export_dir=save_dir,
    dataset_type=fo.types.COCODetectionDataset,
    label_field="predictions",
)

Directory 'GoogleNet' already exists; export will be merged with existing files


 100% |█████████████████| 500/500 [10.5s elapsed, 0s remaining, 46.7 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [10.5s elapsed, 0s remaining, 46.7 samples/s]      
